# Merge Chemical and Production Datasets

Combines cleaned chemical analysis (248 samples) with production metrics (280 monthly records)
on Zafra + Ingenio + Mes keys.

**Source files:**
- Chemical: `analysis/chemical-analysis/data/cleaned/Base_Miel_Final_Filtrado.xlsx`
- Production: `analysis/production-analysis/data/cleaned/Consolidado Productividad_Filtrado.xlsx`

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

## 1. Load cleaned datasets

In [ ]:
# Chemical analysis - cleaned
chem_path = Path('chemical-analysis/data/cleaned/Base_Miel_Final_Filtrado.xlsx')
df_chem = pd.read_excel(chem_path)

print(f"Chemical data shape: {df_chem.shape}")
print(f"Columns: {list(df_chem.columns[:10])}...")  # First 10 cols
print(f"\nKey columns: Zafra, Ingenio, Mes, Muestra")
print(df_chem[['Zafra', 'Ingenio', 'Mes', 'Muestra']].head(2))

In [ ]:
# Production analysis - cleaned
prod_path = Path('production-analysis/data/cleaned/Consolidado Productividad_Filtrado.xlsx')
df_prod = pd.read_excel(prod_path)

print(f"Production data shape: {df_prod.shape}")
print(f"Columns: {list(df_prod.columns)}")
print(f"\nFirst few rows:")
print(df_prod.head(2).to_string())

## 2. Standardize column names for merge

Production data uses uppercase column names: INGENIO, MES_NOMBRE, ZAFRA
Chemical data uses title case: Ingenio, Mes, Zafra

We'll standardize production to match chemical.

In [ ]:
# Rename production columns to match chemical dataset
df_prod.rename(columns={
    'INGENIO': 'Ingenio',
    'MES_NOMBRE': 'Mes',
    'ZAFRA': 'Zafra'
}, inplace=True)

print("Renamed columns in production data")
print(f"New columns: {list(df_prod.columns)}")

In [ ]:
# Normalize case and whitespace for matching
df_chem['Ingenio'] = df_chem['Ingenio'].str.upper().str.strip()
df_prod['Ingenio'] = df_prod['Ingenio'].str.upper().str.strip()

df_chem['Zafra'] = df_chem['Zafra'].astype(str).str.strip()
df_prod['Zafra'] = df_prod['Zafra'].astype(str).str.strip()

df_chem['Mes'] = df_chem['Mes'].str.lower().str.strip()
df_prod['Mes'] = df_prod['Mes'].str.lower().str.strip()

print("Standardized column values")
print(f"\nChemical - Unique Ingenios: {sorted(df_chem['Ingenio'].unique())}")
print(f"Production - Unique Ingenios: {sorted(df_prod['Ingenio'].unique())}")
print(f"\nChemical - Unique Meses: {sorted(df_chem['Mes'].unique())}")
print(f"Production - Unique Meses: {sorted(df_prod['Mes'].unique())}")

## 3. Merge on Zafra + Ingenio + Mes

Using left join on chemical data (248 samples) to keep all honey samples
and add production metrics where available.

In [ ]:
# Merge on Zafra, Ingenio, and Mes
merge_keys = ['Zafra', 'Ingenio', 'Mes']

df_merged = pd.merge(
    df_chem,
    df_prod,
    on=merge_keys,
    how='left',  # Keep all chemical samples
    indicator=True
)

print(f"Merged dataset shape: {df_merged.shape}")
print(f"\nMerge result distribution:")
print(df_merged['_merge'].value_counts())
print(f"\nMatched records: {(df_merged['_merge'] == 'both').sum()}")
print(f"Unmatched (chemistry only): {(df_merged['_merge'] == 'left_only').sum()}")

## 4. Inspect unmatched records

In [ ]:
# Check which chemical samples didn't match production data
unmatched = df_merged[df_merged['_merge'] == 'left_only'][['Zafra', 'Ingenio', 'Mes', 'Muestra']]

if len(unmatched) > 0:
    print(f"Unmatched chemical samples ({len(unmatched)}):")
    print(unmatched.to_string())
else:
    print("✓ Perfect match! All chemical samples have production data.")

## 5. Clean and organize final dataset

In [ ]:
# Drop the merge indicator column
df_merged = df_merged.drop('_merge', axis=1)

# Identify key and data columns
key_cols = ['Zafra', 'Ingenio', 'Mes', 'Muestra', 'NoM', 'Código_ING']
key_cols = [c for c in key_cols if c in df_merged.columns]  # Only if they exist

# Reorder: keys first, then other columns in original order
data_cols = [c for c in df_merged.columns if c not in key_cols]
df_merged = df_merged[key_cols + data_cols]

print(f"Final merged dataset shape: {df_merged.shape}")
print(f"Columns: {len(df_merged.columns)}")
print(f"\nFirst 6 columns: {list(df_merged.columns[:6])}")

## 6. Summary statistics

In [ ]:
print(f"\n" + "="*70)
print(f"MERGE SUMMARY")
print("="*70)
print(f"Chemical samples: {len(df_chem)}")
print(f"Production records: {len(df_prod)}")
print(f"Merged rows: {len(df_merged)}")

# Breakdown by mill
print(f"\nBreakdown by mill:")
prod_cols = [c for c in df_prod.columns if c not in merge_keys]
if prod_cols:
    summary = df_merged.groupby('Ingenio').agg({
        'Muestra': 'count',
        prod_cols[0]: 'count'
    })
    summary.columns = ['Chemical Samples', 'With Production Data']
    summary['Match %'] = (summary['With Production Data'] / summary['Chemical Samples'] * 100).round(1)
    print(summary)
else:
    print(df_merged['Ingenio'].value_counts())

print(f"\n" + "="*70)

## 7. Export to Base Maestra

In [ ]:
output_path = Path('../../Base maestra.xlsx')
df_merged.to_excel(output_path, index=False, sheet_name='Base Maestra')

print(f"✓ Successfully saved to: {output_path}")
print(f"  Rows: {len(df_merged)}")
print(f"  Columns: {len(df_merged.columns)}")

## 8. Preview final dataset

In [ ]:
print(f"\nFirst 3 rows of merged Base Maestra:")
print(df_merged.head(3).to_string())